In [1]:
## Some basic imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [12]:
## Loading the datasets

dallas = pd.read_json('../Demographic/Dallas.json/Dallas.json')
losAngeles = pd.read_json('../Demographic/Los Angeles.json/Los Angeles.json')
philadelphia = pd.read_json('../Demographic/Philadelphia.json/Philadelphia.json')

datasets = [dallas, losAngeles, philadelphia]



               city        lng        lat  \
0            Dallas -97.167352  32.607739   
1            Dallas -96.718005  32.902413   
2            Dallas -97.017309  32.781689   
3            Dallas -96.854292  32.679362   
4            Dallas -96.770589  32.985323   
...             ...        ...        ...   
14045  Philadelphia -75.013961  40.145081   
14046  Philadelphia -74.902059  40.103422   
14047  Philadelphia -75.188758  40.000036   
14048  Philadelphia -75.299942  39.956607   
14049  Philadelphia -75.140953  39.963814   

                                            demographics  
0      {'commute_mode': {'type': 'percent', 'variable...  
1      {'commute_mode': {'type': 'percent', 'variable...  
2      {'commute_mode': {'type': 'percent', 'variable...  
3      {'commute_mode': {'type': 'percent', 'variable...  
4      {'commute_mode': {'type': 'percent', 'variable...  
...                                                  ...  
14045  {'commute_mode': {'type': 'percent', 'v

In [ ]:
## TODO define the predictors and target vars
from sklearn.model_selection import train_test_split

def getTrainSplit(dataset):
   X = pd.DataFrame(dataset)
   Y = dataset
   temp_X_train, temp_X_test, temp_Y_train, temp_Y_test = train_test_split(X, Y)
   return temp_X_train, temp_X_test, temp_Y_train, temp_Y_test

In [ ]:
# Split into training sets 

X_train, X_test, Y_train, Y_test = pd.DataFrame()
for dataset in datasets:
    temp_X_train, temp_X_test, temp_Y_train, temp_Y_test = getTrainSplit(dataset)

    X_train = pd.concat([X_train, temp_X_train], ignore_index=True)
    X_test = pd.concat([X_test, temp_X_test], ignore_index=True)
    Y_train = pd.concat([Y_train, temp_Y_train], ignore_index=True)
    Y_test = pd.concat([Y_test, temp_Y_test], ignore_index=True)



In [14]:
# Start putting together the neural network 
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import Input

In [ ]:
# Declare the classifier
classifier = Sequential()

In [ ]:
# TODO - Add the incoming shape
classifier.add(Input(shape=4))
classifier.add(Dense(units=11, activation='relu'))
classifier.add(Dense(units=11, activation='relu'))
classifier.add(Dense(units=1))

classifier.summary()

In [ ]:
# Compile the model
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# Train the model
classifier.fit(X_train, Y_train, batch_size = 10, nb_epoch = 100)

In [ ]:
result = classifier.predict(X_test)